In [1]:
from typing import Any, List, Optional, Tuple

import os
import sys


from omegaconf import OmegaConf, DictConfig
import hydra
import pyrootutils
from pytorch_lightning.loggers import Logger
import torchaudio


from hydra import initialize, compose



In [2]:
os.environ["PROJECT_ROOT"] = "/Users/christophe/Sites/lightning-hydra-template"

project_root = "/Users/christophe/Sites/lightning-hydra-template"  # Replace with your project path
sys.path.append(project_root)

In [3]:
initialize(config_path="../configs", version_base="1.2")

hydra.initialize()

In [4]:
cfg = compose(config_name="daic_train")

print(OmegaConf.to_yaml(cfg))


task_name: train
tags:
- dev
train: true
test: true
save_state_dict: true
ckpt_path: null
seed: 42
name: lightning-template-0.1.0
datamodule:
  _target_: src.datamodules.daic_datamodules.DaicDataModule
  datasets:
    _target_: src.datamodules.daic_datasets.DaicDataset
    train_val_test_split:
    - 55000
    - 5000
    - 10000
    seed: 42
    json_path: ${paths.data_dir}/daic-woz/annotation.json
    data_path: ${paths.data_dir}/daic-woz/raw/
    label_type: torch.FloatTensor
  loaders:
    train:
      batch_size: 8
      shuffle: true
      num_workers: 8
      drop_last: true
      pin_memory: true
    valid:
      batch_size: 8
      shuffle: false
      num_workers: 8
      drop_last: false
      pin_memory: true
    test:
      batch_size: 8
      shuffle: false
      num_workers: 8
      drop_last: false
      pin_memory: true
    predict:
      batch_size: 8
      shuffle: false
      num_workers: 8
      drop_last: false
      pin_memory: true
  transforms:
    order:
    - 

In [5]:
torchaudio.list_audio_backends()

['soundfile']

In [6]:
datamodule = hydra.utils.instantiate(cfg.datamodule, _recursive_=False)

datamodule.prepare_data()
datamodule.setup('fit')

train_dataloader = datamodule.train_dataloader()

train_dataset = train_dataloader.dataset

item = train_dataloader.dataset.__getitem__(0)

item

ValueError: Sum of input lengths does not equal the length of the input dataset!